In [1]:
import pandas as pd
df = pd.read_table("guillaume.txt", sep=";")
df.head()

C:\Users\spica\AppData\Local\Temp\ipykernel_23040\2198844990.py:2: DtypeWarning: Columns (1,2,5,6,7,8,9,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table("guillaume.txt", sep=";")


,ZIBZIN,IDAvisAutorisationCheque,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,...,TauxImpNb_RB,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure
0,A013010004908126703060931,78643044,0,20,2017-02-01 07:32:14,1,0,0,0,551,...,"37,186667890919111","52,076033757361408",0,1,4,4,4,20,0,27134
1,A013011306908024927155000,78643045,0,20,2017-02-01 07:43:37,1,0,0,0,551,...,"48,844716275908937","52,076033757361408",1,2,"1,7976851851851852",4,4,"28,609999999999999","8,6099999999999994",27817
2,A013010002908283134592527,78643046,0,"57,640000000000001",2017-02-01 07:47:38,1,0,0,0,549,...,"73,118279569892479","52,076033757361408",0,1,4,4,4,"57,640000000000001",0,28058
3,A011010002908105209831316,78643047,0,"54,289999999999999",2017-02-01 07:48:48,0,1,1,1,267,...,"110,05692599620494","53,554233554497365",0,1,4,4,4,"54,289999999999999",0,28128
4,A013010041908000125652029,78643048,0,"26,899999999999999",2017-02-01 08:13:27,1,0,0,0,549,...,"45,36831264567185","52,076033757361408",1,1,"1,9971064814814814",4,4,"59,149999999999999","32,25",29607


In [2]:
len(df)

4646774

# Nettoyage 

On supprime la ligne 1956360 qui comporte comme valeur pour chaque colonne l'en-tête

In [3]:
df.drop(1956360, axis=0, inplace=True)

On change le type des variables quantitatives

In [4]:
var_quanti = ["Montant","ScoringFP1","ScoringFP2","ScoringFP3","TauxImpNb_RB","TauxImpNB_CPM","DiffDateTr1","DiffDateTr2","DiffDateTr3","CA3TRetMtt","CA3TR"]
var_quanti_bug = ["VerifianceCPT1","VerifianceCPT2","VerifianceCPT3","EcartNumCheq","NbrMagasin3J","D2CB"]
new_df=df

for i in var_quanti_bug:
    new_df[i] = pd.to_numeric(new_df[i], downcast="float")
# On change le type des données
for i in var_quanti:
    new_df[i] = new_df[i].str.replace(",",".")
    new_df[i] = pd.to_numeric(new_df[i], downcast="float")

In [5]:
import datetime  

date = []
heure = []


for i in new_df["DateTransaction"]:
    split = i.split(" ")
    date.append(split[0])
    heure.append(split[1])

for i in range(0,len(date)):
    date[i] = datetime.datetime.strptime(date[i], '%Y-%m-%d')



In [6]:
new_df["DateTransaction"]=date
new_df["Heure"]=heure

On s'occupe des des valeurs manquantes 

In [7]:
print(new_df.isna().sum())

ZIBZIN                      0
IDAvisAutorisationCheque    0
FlagImpaye                  0
Montant                     0
DateTransaction             0
CodeDecision                0
VerifianceCPT1              0
VerifianceCPT2              0
VerifianceCPT3              0
D2CB                        0
ScoringFP1                  0
ScoringFP2                  0
ScoringFP3                  0
TauxImpNb_RB                0
TauxImpNB_CPM               0
EcartNumCheq                0
NbrMagasin3J                0
DiffDateTr1                 0
DiffDateTr2                 0
DiffDateTr3                 0
CA3TRetMtt                  0
CA3TR                       0
Heure                       0
dtype: int64


Aucune valeur manquantes

On nettoie la colonne FlagImpaye

In [8]:
new_df.FlagImpaye.value_counts()

0    4584151
0      32627
1      29855
1        140
Name: FlagImpaye, dtype: int64

In [9]:
new_df.FlagImpaye=new_df.FlagImpaye.replace(0,"0")
new_df.FlagImpaye=new_df.FlagImpaye.replace(1,"1")
new_df.FlagImpaye.value_counts(normalize=True)

0    0.993545
1    0.006455
Name: FlagImpaye, dtype: float64

# SAMPLING

## Undersampling

On crée les dataframe train et test 

In [10]:
train_df=new_df[(new_df["DateTransaction"]>='2017-02-01') & (new_df["DateTransaction"]<='2017-08-31')]
test_df=new_df[(new_df["DateTransaction"]>'2017-08-31')]

In [11]:
Y_train = train_df.FlagImpaye
Y_test = test_df.FlagImpaye
X_train = train_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
X_test = test_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])

In [66]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=0.5)
rus.fit(X_train, Y_train)
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)
len(X_train_resampled)


70266

In [98]:
Y_train_resampled.value_counts(normalize=True)

0    0.666667
1    0.333333
Name: FlagImpaye, dtype: float64

# MACHINE LEARNING

# Arbre de décision

In [68]:
import numpy as np

parameters = {'max_depth' : np.arange(start = 16, stop = 24, step = 1) , 
              'min_samples_leaf' : np.arange(start = 2, stop = 4, step = 1),
              'min_samples_split' : np.arange(start = 2, stop = 5, step = 1)}

In [71]:
model_arbre = DecisionTreeClassifier()
f1 = make_scorer(f1_score , average='macro')
model_arbre = GridSearchCV(model_arbre,
                                  parameters,
                                  scoring = f1,
                                  verbose = 2, 
                                  cv = 5)
model_arbre.fit(X_train_resampled, Y_train_resampled)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=   0.8s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=   0.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=   0.8s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=   0.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=   0.8s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=   0.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=4; total time=   0.7s
[CV] END max_depth=16, mi

[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=3; total time=   0.8s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=3; total time=   0.8s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=   0.7s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=   0.8s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=   0.7s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=   0.7s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=   0.8s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   0.8s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   0.8s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   0.7s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   0.7s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   0.7s
[CV]

[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=2; total time=   0.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=2; total time=   0.7s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=2; total time=   0.7s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=   0.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=   0.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=   0.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=   0.7s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=   0.7s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=   0.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=   0.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=   0.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=   0.8s
[CV]

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([16, 17, 18, 19, 20, 21, 22, 23]),
                         'min_samples_leaf': array([2, 3]),
                         'min_samples_split': array([2, 3, 4])},
             scoring=make_scorer(f1_score, average=macro), verbose=2)

In [72]:
print("Voici les paramètres du meilleure modèle : " + 
      str(model_arbre.best_estimator_))
print("Voici le "  + str(model_arbre.scorer_) + 
      " du meilleure modèle : " + str(model_arbre.best_score_))

Voici les paramètres du meilleure modèle : DecisionTreeClassifier(max_depth=23, min_samples_leaf=2)
Voici le make_scorer(f1_score, average=macro) du meilleure modèle : 0.73741331764506


In [74]:
Y_pred = model_arbre.predict(X_test)
Y_pred_proba = model_arbre.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,558936,181902
obs_1,3393,3180


In [75]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,Y_pred))

0.752084194639897


In [76]:
from sklearn.metrics import f1_score
print('f1_score : ' + 
      str(f1_score(Y_test,Y_pred, average='macro')))

f1_score : 0.4454982028156442


## Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

parameters = {'max_depth' : np.arange(start = 16, stop = 24, step = 1) , 
              'min_samples_leaf' : np.arange(start = 2, stop = 4, step = 1),
              'min_samples_split' : np.arange(start = 2, stop = 5, step = 1)}


In [82]:
modele_foret = RandomForestClassifier()
f1 = make_scorer(f1_score , average='macro')
modele_foret = GridSearchCV(modele_foret,
                                  parameters,
                                  scoring = f1,
                                  verbose = 2, 
                                  cv = 5)
modele_foret.fit(X_train_resampled, Y_train_resampled)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=  13.8s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=  14.4s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=  13.1s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=  14.0s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=  12.5s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=  14.6s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=  12.6s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=  14.1s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=  12.1s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=  11.6s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=4; total time=  12.3s
[CV] END max_depth=16, mi

[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=3; total time=  14.7s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=3; total time=  15.7s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=  14.2s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=  13.8s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=  10.2s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=   9.6s
[CV] END max_depth=19, min_samples_leaf=2, min_samples_split=4; total time=  10.0s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   9.0s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   8.8s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   9.0s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   9.3s
[CV] END max_depth=19, min_samples_leaf=3, min_samples_split=2; total time=   8.9s
[CV]

[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=2; total time=  12.0s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=2; total time=  12.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=2; total time=  12.0s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=  11.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=  11.9s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=  11.5s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=  11.8s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=3; total time=  10.4s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=  10.4s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=  10.5s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=  11.1s
[CV] END max_depth=22, min_samples_leaf=3, min_samples_split=4; total time=  10.7s
[CV]

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': array([16, 17, 18, 19, 20, 21, 22, 23]),
                         'min_samples_leaf': array([2, 3]),
                         'min_samples_split': array([2, 3, 4])},
             scoring=make_scorer(f1_score, average=macro), verbose=2)

In [83]:
print("Voici les paramètres du meilleure modèle : " + 
      str(modele_foret.best_estimator_))
print("Voici le "  + str(modele_foret.scorer_) + 
      " du meilleure modèle : " + str(modele_foret.best_score_))

Voici les paramètres du meilleure modèle : RandomForestClassifier(max_depth=23, min_samples_leaf=2, min_samples_split=3)
Voici le make_scorer(f1_score, average=macro) du meilleure modèle : 0.7991936020291229


In [84]:
Y_pred = modele_foret.predict(X_test)
Y_pred_proba = modele_foret.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,592793,148045
obs_1,3012,3561


In [85]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,Y_pred))

0.7978929932794674


In [86]:
from sklearn.metrics import f1_score
print('f1_score : ' + 
      str(f1_score(Y_test,Y_pred, average='macro')))

f1_score : 0.46600635734800766


## Adaboost

In [87]:
from sklearn.ensemble import AdaBoostClassifier

model_adaboost = AdaBoostClassifier()
model_adaboost.fit(X_train_resampled, Y_train_resampled)

AdaBoostClassifier()

In [88]:
Y_pred = model_adaboost.predict(X_test)
Y_pred_proba = model_adaboost.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,642046,98792
obs_1,3565,3008


In [89]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,Y_pred))

0.8630512529250974


In [90]:
from sklearn.metrics import f1_score
print('f1_score : ' + 
      str(f1_score(Y_test,Y_pred, average='macro')))

f1_score : 0.4908426244342296


## SVM

In [91]:
from sklearnex import patch_sklearn
patch_sklearn("SVC")

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [92]:
from sklearn.svm import SVC
model_svm = SVC()
model_svm.fit(X_train_resampled, Y_train_resampled)

SVC()

In [93]:
print(model_svm.n_support_)

[23429 23416]


In [94]:
Y_pred = model_svm.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,740408,430
obs_1,6564,9


In [95]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,Y_pred))

0.990642364107566


In [96]:
from sklearn.metrics import f1_score
print('f1_score : ' + 
      str(f1_score(Y_test,Y_pred, average='macro')))

f1_score : 0.4989330794447502


## Bagging

In [101]:
from sklearn.ensemble import BaggingClassifier
model_bagging = BaggingClassifier(SVC())

model_bagging.fit(X_train_resampled, Y_train_resampled)

BaggingClassifier(base_estimator=SVC())

In [102]:
Y_pred = model_bagging.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,740518,320
obs_1,6565,8


In [103]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,Y_pred))

0.9907882008694012


In [104]:
from sklearn.metrics import f1_score
print('f1_score : ' + 
      str(f1_score(Y_test,Y_pred, average='macro')))

f1_score : 0.49884562138386324
